In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [75]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ntcv4gk9
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ntcv4gk9
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=2de32239cc08bae9d76a3cdcfcff83ecec5f2264490721ac25caf47a96b7aca7
  Stored in directory: /tmp/pip-ephem-wheel-cache-25uusayr/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [77]:
%%cu
// Problem 1
#include <iostream>
using namespace std;

const short N = 16;

__managed__ int a[N];

__global__ void sum() {
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads)
		{
			const auto st = tid * step_size * 2;
			const auto en = st + step_size;
      if (st < N && en < N)
			a[st] += a[en];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
}

int main() {
  int n = N;
  if ((n != 1) && (n & (n - 1))) {
      n |= n >> 1;
      n |= n >> 2;
      n |= n >> 4;
      n |= n >> 8;
      n |= n >> 16;
      n = n + 1;
  }

	for (int i = 0; i < N; i++) a[i] = 1;
	sum <<<1, n / 2 >>>();
  cudaDeviceSynchronize();
  int result = a[0];
	cout << "Sum is " << result << endl;
	return 0;
}

Sum is 16



In [9]:
%%cu
// Problem 2
#include <stdio.h>

const short N = 10;

__managed__ int a[N], b[N], c[N];

__global__ void Vector_Addition () {
      //Get the id of thread within a block
      
      unsigned short tid = blockDim.x * blockIdx.x + threadIdx.x;

     // check the boundry condition for the threads
      if (tid < N) c[tid] = a[tid] + b[tid];

}


int main (){
      for (int i = 0; i < N; i++){
            a[i] = -i ;
            b[i] = i*i ; 
      }
      int threadsPerBlock = 256;
      int blocksPerGrid =(N + threadsPerBlock - 1) / threadsPerBlock;
      Vector_Addition <<<blocksPerGrid, threadsPerBlock>>> ( ) ;
      cudaDeviceSynchronize();

      for ( int i = 0; i<N; i++ )
      printf("%d + %d = %d\n", a[i] , b[i] , c[i]);

      return 0 ;
}


0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 2
-3 + 9 = 6
-4 + 16 = 12
-5 + 25 = 20
-6 + 36 = 30
-7 + 49 = 42
-8 + 64 = 56
-9 + 81 = 72



In [72]:
%%cu
// Problem 3
#include <iostream>
using namespace std;

const int m = 3, n = 2, p = 4;
__managed__ int a[m * n], b[n * p], c[m * p];

__global__ void MatrixMultiply() {
  int row = blockIdx.x * p + threadIdx.x;
  int col = blockIdx.y * m + threadIdx.y;
  
  if(row < m && col < p) {
    //printf("%d %d\n", row, col);
    float tem = 0;
    for(int k = 0; k < n; k++) {
        tem += a[row * n + k] * b[k * p + col];
    }
    c[row * p + col] = tem;
  }    
}

int main() {
  for (int i = 0; i < m * n; i++) a[i] = rand() % 50;  
  for (int i = 0; i < n * p; i++) b[i] = rand() % 50;
  cout << "Matrix A:\n";
  for (int i = 0; i < m; i++) {
      for (int j = 0; j < n; j++) cout << a[i * n + j] << ' ';
      cout << '\n';
  }  
  cout << "Matrix B:\n";
  for (int i = 0; i < n; i++) {
      for (int j = 0; j < p; j++) cout << b[i * p + j] << ' ';
      cout << '\n';
  } 
  dim3 threadsPerBlock(m, p);
  dim3 blocksPerGrid(1, 1);
  if (m * p > 512){
    threadsPerBlock.x = 512;
    threadsPerBlock.y = 512;
    blocksPerGrid.x = ceil(double(m)/double(threadsPerBlock.x));
    blocksPerGrid.y = ceil(double(p)/double(threadsPerBlock.y));
  }
  MatrixMultiply<<<blocksPerGrid,threadsPerBlock>>>();
  cudaDeviceSynchronize();
  cout << "Matrix C:\n";
  for (int i = 0; i < m; i++) {
      for (int j = 0; j < p; j++) cout << c[i * p + j] << ' ';
      cout << '\n';
  } 
}

Matrix A:
33 36 
27 15 
43 35 
Matrix B:
36 42 49 21 
12 27 40 9 
Matrix C:
1620 2358 3057 1017 
1152 1539 1923 702 
1968 2751 3507 1218 



In [112]:
%%cu
// Problem 5
#include<iostream>
using namespace std;

__managed__ int N = 10, E = 0;
// Kernel definition
__global__ void BFS(int* DV_a, int* DE_a, int* DC, bool* DF, bool* DX) {
    
    int tid = threadIdx.x;
    if(tid < N) {
        if(DF[tid]) {
            DF[tid] = false, DX[tid] = true;
            int st = DV_a[tid], en = 2 * E;
            //printf("%d %d\n", st, en);
            if(tid < N - 1) en = DV_a[tid + 1];
            for(int i = st; i < en; i++) {
                int nid = DE_a[i];
                if(!DX[nid]) {
                    DC[nid] = DC[tid] + 1;
                    DF[nid] = true;
                }
            }
        }
    }
}

int main() {
  int G[N][N];
  // Generating Graph
  for (int i = 0; i < N; i++) {
      for (int j = i + 1; j < N; j++) {
          if (i == j) G[i][j] = 0;
          else G[i][j] = G[j][i] = rand() % 2, E += G[i][j];
      }
  }    
  int V_a[N], E_a[2 * E];
  int t = 0;
  for (int i = 0; i < N; i++) {
      V_a[i] = t;
      for (int j = 0; j < N; j++) {
          if (G[i][j]) E_a[t] = j, t++;
      }
  }

  int C[N]; bool F[N], X[N];
  for(int i = 0; i < N; i++) {
      C[i] = INT_MAX;
      F[i] = X[i] = false;
  }
  // source
  int s = 2;
  C[s] = 0, F[s] = true;
  cout << E << '\n';
  int *DV_a, *DE_a, *DC;
  bool *DF, *DX;
  cudaMalloc((void**) & DV_a, N * sizeof(int));
  cudaMalloc((void**) & DE_a, 2 * E * sizeof(int));
  cudaMalloc((void**) & DC, N * sizeof(int));
  cudaMalloc((void**) & DF, N * sizeof(bool));
  cudaMalloc((void**) & DX, N * sizeof(bool));
    
  cudaMemcpy(DV_a, V_a, N * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(DE_a, E_a, 2 * E * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(DC, C, N * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(DF, F, N * sizeof(bool), cudaMemcpyHostToDevice);
  cudaMemcpy(DX, X, N * sizeof(bool), cudaMemcpyHostToDevice);
  int u = 10;
  while (1) {
      int ok = 0;
      BFS<<<1, N>>>(DV_a, DE_a, DC, DF, DX);
      cudaDeviceSynchronize();
      cudaMemcpy(F, DF, N * sizeof(bool), cudaMemcpyDeviceToHost);
      for(int i = 0; i < N; i++) {
          if(F[i]) {
              ok = 1;
              break;
          }
      }
      if(!ok) break;
    }
    cudaMemcpy(C, DC, N * sizeof(int), cudaMemcpyDeviceToHost);
    cout << "Distances: \n";
    for (auto to : C) cout << to << ' ';
    cudaFree(DV_a);
    cudaFree(DE_a);
    cudaFree(DC);
    cudaFree(DF);
    cudaFree(DX);
    return 0;
}


25
Distances: 
2 2 0 3 3 2 1 3 2 2 


In [74]:
%%cu
// Problem 7
#include <iostream>
#include <algorithm>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sort.h>
#include<thrust/copy.h>
using namespace std;

const int N = 17;

int main() {
  thrust::host_vector<int> h(N);
  for (int i = 0; i < N; i++) h[i] = rand() % 50;
  for (auto to : h) cout << to << ' ';
  cout << '\n';
  thrust::device_vector<int> d = h;
  thrust::sort(d.begin(), d.end());
  thrust::copy(d.begin(), d.end(), h.begin());
  for (auto to : h) cout << to << ' ';
  cout << '\n';
  return 0;
}

33 36 27 15 43 35 36 42 49 21 12 27 40 9 13 26 40 
9 12 13 15 21 26 27 27 33 35 36 36 40 40 42 43 49 

